## L1
## Goal 1: Rule-based, no external knowledge

In [5]:
def extract_entities(tokens):
    entities = []
    prev_drug = ""
    
    for i in range(len(tokens)):
        token = tokens[i]
        word = token[0]
        
        # Prefix and sufix rules
        drug_prefixes = ('pheny', 'digox', 'warfa', 'meth', 'theophy', 'lith', 'keto', 'cime', 'insu', 'fluox', 'alcoh', 'cyclos', 'eryth', 'carba', 'rifa', 'caffe')
        drug_sufixes = ('pitant', 'dine', 'azole', 'mide', 'pine', 'line', 'mine', 'tine', 'arin', 'avir', 'azem', 'rine', 'rone', 'arbital', 'olol', 'afil', 'inol', 'zolam')
        
        group_prefixes = ('benzo', 'beta', 'antico', 'antide', 'antibi', 'antihi', 'nsai', 'contra')
        group_sufixes = ('steroids','tics', 'ants', 'ents', 'tors', 'acid', 'acids', 'ceptives', 'gens', 'pines', 'lines', 'mines')
        
        brand_prefixes = ('aspi', 'accu', 'beza', 'star', 'exja')
        brand_sufixes = ('tane', 'dine', 'anil')
        
        drug_n_prefixes = ('ibog', 'endo')
        drug_n_sufixes = ('ate', 'sin', 'toxin', 'orfon')
        
        if word.lower().startswith(drug_prefixes) or word.lower().endswith(drug_sufixes):
            entities.append({'name':word, 'offset': str(token[1])+'-'+str(token[2]),'type':'drug'})
            prev_drug = str(word)+ " "+str(token[1])+" "+"drug"
            
        elif word.lower().startswith(group_prefixes) or word.lower().endswith(group_sufixes):
            if prev_drug != '':
                info = prev_drug.split(" ")
                if len(entities) > 0 and info[2]=='group':
                    entities.pop()
                    entities.append({'name':str(info[0])+word, 'offset': str(info[1])+'-'+str(token[2]),'type':'group'})
            else:
                entities.append({'name':word, 'offset': str(token[1])+'-'+str(token[2]),'type':'group'})
            prev_drug = str(word)+ " "+str(token[1])+ " "+"group"
        
        elif (word.isupper() and len(word)>4) or word.lower().startswith(brand_prefixes) or word.lower().endswith(brand_sufixes):
            entities.append({'name':word, 'offset': str(token[1])+'-'+str(token[2]),'type':'brand'})
            prev_drug = str(word)+ " "+str(token[1])+ " "+ "brand"
        
        elif word.isupper() or word.lower().startswith(drug_n_prefixes) or word.lower().endswith(drug_n_prefixes)\
        or (bool(re.search(r'\d', word)) and '-' in word):
            entities.append({'name':word, 'offset': str(token[1])+'-'+str(token[2]),'type':'drug_n'})
            prev_drug = str(word)+ " "+str(token[1])+ " "+"drug_n"
        else:
            prev_drug = ""
        
    return entities

### Comments


SCORES FOR THE GROUP: develGoal RUN=1

Strict matching (boundaries + type)
cor	inc	par	mis	spu	total	prec	recall	F1
773	292	0	706	496	1771	0.5	0.44	0.46



Exact matching
cor	inc	par	mis	spu	total	prec	recall	F1
815	250	0	706	496	1771	0.52	0.46	0.49



Partial matching
cor	inc	par	mis	spu	total	prec	recall	F1
815	0	250	706	496	1771	0.52	0.53	0.53



type matching
cor	inc	par	mis	spu	total	prec	recall	F1
904	161	0	706	496	1771	0.58	0.51	0.54



SCORES FOR ENTITY TYPE
Exact matching on drug
cor	inc	par	mis	spu	total	prec	recall	F1
511	14	0	520	45	1045	0.9	0.49	0.63


Exact matching on brand
cor	inc	par	mis	spu	total	prec	recall	F1
115	0	0	65	25	180	0.82	0.64	0.72


Exact matching on group
cor	inc	par	mis	spu	total	prec	recall	F1
120	117	0	217	83	454	0.38	0.26	0.31


Exact matching on drug_n
cor	inc	par	mis	spu	total	prec	recall	F1
27	0	0	65	33	92	0.45	0.29	0.36


MACRO-AVERAGE MEASURES:
P	R	F1
0.64	0.42	0.5

## Goal 2 : Rule-based, using external knowledge

In [3]:
def extract_entities(tokens):
    entities = []
    
    for i in range(len(tokens)):
        token = tokens[i]
        word = token[0]
        
        drug_n_prefixes = ('18-m', 'ibog', 'endo', 'toxi')
        drug_n_sufixes = ('ine', 'ate', '8-mc', 'sin', 'xin', 'pge2', 'mhd')
        
        if word.lower() in drug_bank:
            entities.append({'name':word, 'offset': str(token[1])+'-'+str(token[2]),'type':drug_bank[word.lower()]})
        
        
        elif word.isupper() or word.lower().startswith(drug_n_prefixes) or word.lower().endswith(drug_n_prefixes):
            entities.append({'name':word, 'offset': str(token[1])+'-'+str(token[2]),'type':'drug_n'})
            
        else:
            for j in range(1, 5):
                if i>=j-1:
                    words_joined  = ' '.join([t[0] for t in tokens[i-j:i+1]])   
                    if words_joined.lower() in drug_bank:
                        entities.append({'name':words_joined, 'offset': str(tokens[i-j][1])+'-'+str(tokens[i][2]),'type':drug_bank[words_joined.lower()]})
    return entities


### Comments


SCORES FOR THE GROUP: develGoal2 RUN=1

Strict matching (boundaries + type)
cor	inc	par	mis	spu	total	prec	recall	F1
1177	350	0	244	524	1771	0.57	0.66	0.62



Exact matching
cor	inc	par	mis	spu	total	prec	recall	F1
1270	257	0	244	524	1771	0.62	0.72	0.66



Partial matching
cor	inc	par	mis	spu	total	prec	recall	F1
1270	0	257	244	524	1771	0.62	0.79	0.69



type matching
cor	inc	par	mis	spu	total	prec	recall	F1
1291	236	0	244	524	1771	0.63	0.73	0.68



SCORES FOR ENTITY TYPE
Exact matching on drug
cor	inc	par	mis	spu	total	prec	recall	F1
866	44	0	135	76	1045	0.88	0.83	0.85


Exact matching on brand
cor	inc	par	mis	spu	total	prec	recall	F1
170	3	0	7	32	180	0.83	0.94	0.88


Exact matching on group
cor	inc	par	mis	spu	total	prec	recall	F1
115	52	0	287	52	454	0.53	0.25	0.34


Exact matching on drug_n
cor	inc	par	mis	spu	total	prec	recall	F1
26	15	0	51	37	92	0.33	0.28	0.31


MACRO-AVERAGE MEASURES:
P	R	F1
0.64	0.58	0.6

## Goal 4 : ML, no external knowledge

### Comments




## Goal 4 : ML, using external knowledge